In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyqubo --quiet
!pip install dwave-ocean-sdk --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.8/256.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.5/167.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00


In [ ]:
from pyqubo import Binary
from pyqubo import Spin
from pyqubo import Array
import neal
from dwave.samplers import SimulatedAnnealingSampler
import dimod
from math import *

import sympy
from collections import defaultdict
import numpy as np

Old quadrizate function

In [ ]:
# @title
def n3(a,b,c,bqm,n,interaction):
  bqm.add_variable('w'+str(n),-2*interaction)
  bqm.add_interaction('w'+str(n),a,interaction)
  bqm.add_interaction('w'+str(n),b,interaction)
  bqm.add_interaction('w'+str(n),c,interaction)
  return bqm

def n4(a,b,c,d,bqm,n,interaction):
  bqm.add_variable('w'+str(n),-3*interaction)
  bqm.add_interaction('w'+str(n),a,interaction)
  bqm.add_interaction('w'+str(n),b,interaction)
  bqm.add_interaction('w'+str(n),c,interaction)
  bqm.add_interaction('w'+str(n),d,interaction)
  return bqm

def p3(a,b,c,bqm,n,interaction):
  bqm.add_variable('w'+str(n),-1*interaction)
  bqm.offset+=(interaction)
  bqm.add_interaction('w'+str(n),a,interaction)
  bqm.add_interaction('w'+str(n),b,interaction)
  bqm.add_interaction('w'+str(n),c,interaction)
  if a in bqm.linear.keys():
    bqm.linear[a]-=interaction
  else:
    bqm.linear[a]=-interaction
  if b in bqm.linear.keys():
    bqm.linear[b]-=interaction
  else:
    bqm.linear[b]=-interaction
  if c in bqm.linear.keys():
    bqm.linear[c]-=interaction
  else:
    bqm.linear[c]=-interaction
  for i in bqm.quadratic.keys():
    if a in i and b in i:
      bqm.quadratic[i]+=interaction
      break
  else:
    bqm.quadratic[(a,b)]=interaction
  for i in bqm.quadratic.keys():
    if a in i and c in i:
      bqm.quadratic[i]+=interaction
      break
  else:
    bqm.quadratic[(a,c)]=interaction
  for i in bqm.quadratic.keys():
    if b in i and c in i:
      bqm.quadratic[i]+=interaction
      break
  else:
    bqm.quadratic[(b,c)]=interaction
  return bqm

def p4(a,b,c,d,bqm,n,interaction):
  bqm.add_variable('w'+str(n),-2*interaction)
  bqm.offset+=(2*interaction)
  bqm.add_interaction('w'+str(n),d,4*interaction)
  if d in bqm.linear.keys():
    bqm.linear[d]-=2*interaction
  else:
    bqm.linear[d]=-2*interaction
  bqm=p3(a,b,c,bqm,str(n)+'#',interaction)
  bqm=n4(a,b,c,'w'+str(n),bqm,str(n)+'##',-1*interaction)
  return bqm


def quadrizate(bqm):
    #Linear
    del_list=[]
    testset=[]
    intdict={}
    for i in bqm.linear.keys():
        if ' * ' in i:
            del_list.append(i)
            parts = i.split(' * ')
            if len(parts) == 3:
                a, b, c = parts
                uniqcheck = sorted({a, b, c})
                if uniqcheck in testset:
                    intdict[tuple(uniqcheck)] += bqm.linear[i]
                else:
                    testset.append(uniqcheck)
                    intdict[tuple(uniqcheck)] = bqm.linear[i]
                continue

            a,b=parts
            for j in bqm.quadratic.keys():
                if a in j and b in j:
                    bqm.quadratic[j]+=bqm.linear[i]
                    break
            else:
                bqm.quadratic[(a,b)]=bqm.linear[i]

    #Quadratic
    for i in list(bqm.quadratic.keys()):

        # --- MODIFICATION IS INSIDE THIS BLOCK ---
        if ' * ' in i[0] and ' * ' in i[1]:
            parts_a = i[0].split(' * ')
            parts_b = i[1].split(' * ')

            # Original logic for the standard ('p1*q1', 'p2*q2') case
            if len(parts_a) == 2 and len(parts_b) == 2:
                a,b=parts_a
                c,d=parts_b
                uniqcheck={a,b,c,d}
                uniqcheck=sorted(uniqcheck)
                if uniqcheck in testset and len(uniqcheck)!=2:
                    intdict[tuple(uniqcheck)]+=bqm.quadratic[i]
                    continue
                else:
                    testset.append(uniqcheck)
                    intdict[tuple(uniqcheck)]=bqm.quadratic[i]
                    continue
            # New logic for complex cases like ('p1*p2*q1', 'p2*q1')
            else:
                all_vars = parts_a + parts_b
                uniqcheck = sorted(set(all_vars)) # Use set to get unique variables
                if uniqcheck in testset:
                    intdict[tuple(uniqcheck)] += bqm.quadratic[i]
                else:
                    testset.append(uniqcheck)
                    intdict[tuple(uniqcheck)] = bqm.quadratic[i]
                continue
        # --- END OF MODIFICATION ---

        elif ' * ' in i[0]:
            parts = i[0].split(' * ')
            if len(parts) == 3:
                a, b, c = parts
                d = i[1]
                uniqcheck = sorted({a, b, c, d})
                if uniqcheck in testset:
                    intdict[tuple(uniqcheck)] += bqm.quadratic[i]
                else:
                    testset.append(uniqcheck)
                    intdict[tuple(uniqcheck)] = bqm.quadratic[i]
                continue

            a,b=parts
            c=i[1]
            uniqcheck={a,b,c}
            uniqcheck=sorted(uniqcheck)
            if len(uniqcheck)!=3:
                a1,b1=list(uniqcheck)
                for j in bqm.quadratic.keys():
                    if a1 in j and b1 in j:
                        bqm.quadratic[j]+=bqm.quadratic[i]
                        break
                else:
                    bqm.quadratic[(a1,b1)]=bqm.quadratic[i]
                continue
            if uniqcheck in testset:
                intdict[tuple(uniqcheck)]+=bqm.quadratic[i]
                continue
            else:
                testset.append(uniqcheck)
                intdict[tuple(uniqcheck)]=bqm.quadratic[i]

        elif ' * ' in i[1]:
            parts = i[1].split(' * ')
            if len(parts) == 3:
                a, b, c = parts
                d = i[0]
                uniqcheck = sorted({a, b, c, d})
                if uniqcheck in testset:
                    intdict[tuple(uniqcheck)] += bqm.quadratic[i]
                else:
                    testset.append(uniqcheck)
                    intdict[tuple(uniqcheck)] = bqm.quadratic[i]
                continue

            a,b=parts
            c=i[0]
            uniqcheck={a,b,c}
            uniqcheck=sorted(uniqcheck)
            if len(uniqcheck)!=3:
                a1,b1=list(uniqcheck)
                for j in bqm.quadratic.keys():
                    if a1 in j and b1 in j:
                        bqm.quadratic[j]+=bqm.quadratic[i]
                        break
                else:
                    bqm.quadratic[(a1,b1)]=bqm.quadratic[i]
                continue
            if uniqcheck in testset:
                intdict[tuple(uniqcheck)]+=bqm.quadratic[i]
                continue
            else:
                testset.append(uniqcheck)
                intdict[tuple(uniqcheck)]=bqm.quadratic[i]

    n=0
    for i in intdict.keys():
        l=list(i)
        if len(l)==3 and intdict[i]>0:
            bqm=p3(l[0],l[1],l[2],bqm,n,intdict[i])
        elif len(l)==3 and intdict[i]<0:
            bqm=n3(l[0],l[1],l[2],bqm,n,intdict[i])
        elif len(l)==4 and intdict[i]>0:
            bqm=p4(l[0],l[1],l[2],l[3],bqm,n,intdict[i])
        elif len(l)==4 and intdict[i]<0:
            bqm=n4(l[0],l[1],l[2],l[3],bqm,n,intdict[i])
        n+=1

    for i in del_list:
        del bqm.linear[i]
    return bqm

New quadrizate function

In [ ]:
# @title
def n3(a,b,c,bqm,n,interaction):
  bqm.add_variable('w'+str(n),-2*interaction)
  bqm.add_interaction('w'+str(n),a,interaction)
  bqm.add_interaction('w'+str(n),b,interaction)
  bqm.add_interaction('w'+str(n),c,interaction)
  return bqm

def n4(a,b,c,d,bqm,n,interaction):
  bqm.add_variable('w'+str(n),-3*interaction)
  bqm.add_interaction('w'+str(n),a,interaction)
  bqm.add_interaction('w'+str(n),b,interaction)
  bqm.add_interaction('w'+str(n),c,interaction)
  bqm.add_interaction('w'+str(n),d,interaction)
  return bqm

def p3(a,b,c,bqm,n,interaction):

  bqm.add_variable('w'+str(n),1*interaction)
  bqm.add_interaction('w'+str(n),c,interaction)
  bqm.add_interaction('w'+str(n),b,-1*interaction)
  bqm.add_interaction('w'+str(n),a,-1*interaction)

  for i in bqm.quadratic.keys():
    if a in i and b in i:
      bqm.quadratic[i]+=interaction
      break
  else:
    bqm.quadratic[(a,b)]=interaction

  return bqm

def p4(a,b,c,d,bqm,n,interaction):
  bqm.add_variable('w'+str(n),interaction)
  bqm.add_variable('w#'+str(n),interaction)

  bqm.add_interaction('w'+str(n),a,-1*interaction)
  bqm.add_interaction('w#'+str(n),d,interaction)
  bqm.add_interaction('w'+str(n),b,-1*interaction)
  bqm.add_interaction('w#'+str(n),c,-1*interaction)
  bqm.add_interaction('w'+str(n),c,interaction)
  bqm.add_interaction('w#'+str(n),'w'+str(n),-1*interaction)

  for i in bqm.quadratic.keys():
    if a in i and b in i:
      bqm.quadratic[i]+=interaction
      break
  else:
    bqm.quadratic[(a,b)]=interaction

  return bqm

def quadrizate(bqm):
    #Linear
    del_list=[]
    testset=[]
    intdict={}
    for i in bqm.linear.keys():
        if ' * ' in i:
            del_list.append(i)
            parts = i.split(' * ')
            if len(parts) == 3:
                a, b, c = parts
                uniqcheck = sorted({a, b, c})
                if uniqcheck in testset:
                    intdict[tuple(uniqcheck)] += bqm.linear[i]
                else:
                    testset.append(uniqcheck)
                    intdict[tuple(uniqcheck)] = bqm.linear[i]
                continue

            a,b=parts
            for j in bqm.quadratic.keys():
                if a in j and b in j:
                    bqm.quadratic[j]+=bqm.linear[i]
                    break
            else:
                bqm.quadratic[(a,b)]=bqm.linear[i]

    #Quadratic
    for i in list(bqm.quadratic.keys()):

        # --- MODIFICATION IS INSIDE THIS BLOCK ---
        if ' * ' in i[0] and ' * ' in i[1]:
            parts_a = i[0].split(' * ')
            parts_b = i[1].split(' * ')

            # Original logic for the standard ('p1*q1', 'p2*q2') case
            if len(parts_a) == 2 and len(parts_b) == 2:
                a,b=parts_a
                c,d=parts_b
                uniqcheck={a,b,c,d}
                uniqcheck=sorted(uniqcheck)
                if uniqcheck in testset and len(uniqcheck)!=2:
                    intdict[tuple(uniqcheck)]+=bqm.quadratic[i]
                    continue
                else:
                    testset.append(uniqcheck)
                    intdict[tuple(uniqcheck)]=bqm.quadratic[i]
                    continue
            # New logic for complex cases like ('p1*p2*q1', 'p2*q1')
            else:
                all_vars = parts_a + parts_b
                uniqcheck = sorted(set(all_vars)) # Use set to get unique variables
                if uniqcheck in testset:
                    intdict[tuple(uniqcheck)] += bqm.quadratic[i]
                else:
                    testset.append(uniqcheck)
                    intdict[tuple(uniqcheck)] = bqm.quadratic[i]
                continue
        # --- END OF MODIFICATION ---

        elif ' * ' in i[0]:
            parts = i[0].split(' * ')
            if len(parts) == 3:
                a, b, c = parts
                d = i[1]
                uniqcheck = sorted({a, b, c, d})
                if uniqcheck in testset:
                    intdict[tuple(uniqcheck)] += bqm.quadratic[i]
                else:
                    testset.append(uniqcheck)
                    intdict[tuple(uniqcheck)] = bqm.quadratic[i]
                continue

            a,b=parts
            c=i[1]
            uniqcheck={a,b,c}
            uniqcheck=sorted(uniqcheck)
            if len(uniqcheck)!=3:
                a1,b1=list(uniqcheck)
                for j in bqm.quadratic.keys():
                    if a1 in j and b1 in j:
                        bqm.quadratic[j]+=bqm.quadratic[i]
                        break
                else:
                    bqm.quadratic[(a1,b1)]=bqm.quadratic[i]
                continue
            if uniqcheck in testset:
                intdict[tuple(uniqcheck)]+=bqm.quadratic[i]
                continue
            else:
                testset.append(uniqcheck)
                intdict[tuple(uniqcheck)]=bqm.quadratic[i]

        elif ' * ' in i[1]:
            parts = i[1].split(' * ')
            if len(parts) == 3:
                a, b, c = parts
                d = i[0]
                uniqcheck = sorted({a, b, c, d})
                if uniqcheck in testset:
                    intdict[tuple(uniqcheck)] += bqm.quadratic[i]
                else:
                    testset.append(uniqcheck)
                    intdict[tuple(uniqcheck)] = bqm.quadratic[i]
                continue

            a,b=parts
            c=i[0]
            uniqcheck={a,b,c}
            uniqcheck=sorted(uniqcheck)
            if len(uniqcheck)!=3:
                a1,b1=list(uniqcheck)
                for j in bqm.quadratic.keys():
                    if a1 in j and b1 in j:
                        bqm.quadratic[j]+=bqm.quadratic[i]
                        break
                else:
                    bqm.quadratic[(a1,b1)]=bqm.quadratic[i]
                continue
            if uniqcheck in testset:
                intdict[tuple(uniqcheck)]+=bqm.quadratic[i]
                continue
            else:
                testset.append(uniqcheck)
                intdict[tuple(uniqcheck)]=bqm.quadratic[i]

    n=0
    for i in intdict.keys():
        l=list(i)
        if len(l)==3 and intdict[i]>0:
            bqm=p3(l[0],l[1],l[2],bqm,n,intdict[i])
        elif len(l)==3 and intdict[i]<0:
            bqm=n3(l[0],l[1],l[2],bqm,n,intdict[i])
        elif len(l)==4 and intdict[i]>0:
            bqm=p4(l[0],l[1],l[2],l[3],bqm,n,intdict[i])
        elif len(l)==4 and intdict[i]<0:
            bqm=n4(l[0],l[1],l[2],l[3],bqm,n,intdict[i])
        n+=1

    for i in del_list:
        del bqm.linear[i]
    return bqm

Direct Multiplication approach

In [ ]:
# @title
def factorize1(N):
  nx=floor(log2(N))-1
  ny=ceil(0.5*log2(N))-1
  x=Array.create('x',shape=nx,vartype='BINARY')
  y=Array.create('y',shape=ny,vartype='BINARY')
  H1=sum([2**(i+1)*x[i] for i in range(nx)])+1
  H2=sum([2**(i+1)*y[i] for i in range(ny)])+1
  H=(N-H1*H2)*(N-H1*H2)
  model=H.compile()
  bqm=model.to_bqm()
  bqm=quadrizate(bqm)
  print("Compiled bqm:",bqm)
  print("BQM has {} non-fixed variables.".format(len(bqm.variables)))
  if len(bqm.variables)>2000:
    print("Not solving ts")
    return None


  sampler = SimulatedAnnealingSampler()
  sampleset = sampler.sample(bqm, num_reads=1000)
  best_solution = sampleset.first
  #print("Best solution:",best_solution.sample)
  print(f"Energy: {best_solution.energy}")
  fx=1
  fy=1
  for i in dict(best_solution.sample).keys():
    if 'x[' in i:
      a=int(i.strip('x[]'))+1
      if dict(best_solution.sample)[i]==1:
        fx+=2**a
    elif 'y[' in i:
      a=int(i.strip('y[]'))+1
      if dict(best_solution.sample)[i]==1:
        fy+=2**a
  print(f"Factors of {N} are:{fx},{fy}")
  return None

In [ ]:
# @title
def get_qubo_matrix1(N):
  nx=floor(log2(N))-1
  ny=ceil(0.5*log2(N))-1
  x=Array.create('x',shape=nx,vartype='BINARY')
  y=Array.create('y',shape=ny,vartype='BINARY')
  H1=sum([2**(i+1)*x[i] for i in range(nx)])+1
  H2=sum([2**(i+1)*y[i] for i in range(ny)])+1
  H=(N-H1*H2)*(N-H1*H2)
  model=H.compile()
  bqm=model.to_bqm()
  bqm=quadrizate(bqm)

  # Convert BQM to QUBO dictionary and index mapping
  qubo_dict, offset = bqm.to_qubo()

  # Create a mapping from variable name to index
  variable_to_index = {v: i for i, v in enumerate(bqm.variables)}
  index_to_variable = {i: v for v, i in variable_to_index.items()}

  # Create the QUBO matrix
  size = len(bqm.variables)
  qubo_matrix = np.zeros((size, size))

  for (u, v), value in qubo_dict.items():
      i = variable_to_index[u]
      j = variable_to_index[v]
      qubo_matrix[i, j] = value

  return qubo_matrix, index_to_variable

In [ ]:
def post_processing1(sol,index_to_variable,N):#Assuming the solution from SPIM is given as a list of 0s and 1s and sol[i] represents the solution of the ith index of qubo matrix
  sol_dict={}
  for i in range(len(sol)):
    sol_dict[index_to_variable[i]]=sol[i]
  fx=1
  fy=1
  for i in sol_dict.keys():
    if 'x[' in i:
      a=int(i.strip('x[]'))+1
      if sol_dict[i]==1:
        fx+=2**a
    elif 'y[' in i:
        a=int(i.strip('y[]'))+1
        if sol_dict[i]==1:
          fy+=2**a
  print(f"Factors of {N} are:{fx},{fy}")
  return None

Column based approach

In [ ]:
# @title
def initialize_variables(N):###
    # Determine the number of bits for P and Q
    n_m = ceil(log2(N + 1))
    #n_p=floor(log2(N))-1
    n_q=ceil(0.5*log2(N))
    n_p=n_q

    print(f"Factoring N = {N} ({n_m} bits)")
    print(f"Assuming n_p = {n_p}, n_q = {n_q}")

    # Create symbolic variables for the unknown bits of P and Q
    # p_0 and p_{n_p-1} are fixed to 1
    p = {i: sympy.symbols(f'p_{i}') for i in range(1, n_p)}
    p[0] = 1
    p[n_p-1]=1 ##

    # q_0 and q_{n_q-1} are fixed to 1
    q = {i: sympy.symbols(f'q_{i}') for i in range(1, n_q)}
    q[0] = 1
    q[n_q-1]=1 ##

    # Create symbolic variables for the carry bits
    # s[j, i] represents a carry from column j to column i
    s = defaultdict(lambda: 0)
    max_sum_terms = n_q + (n_q - 1) # Max terms in a central column  #Waste term
    max_carry_val = max_sum_terms   #Waste term
    for i in range(1, n_p+n_q):
        # Estimate max possible sum in column i to determine needed carries
        num_prod_terms = min(i, n_q - 1) - max(i - n_p + 1, 0) + 1
        num_carry_in = i - 1 # A rough upper bound
        max_sum_in_col = num_prod_terms + num_carry_in

        if max_sum_in_col > 1:
            num_carry_out_bits = floor(log2(max_sum_in_col))
            for j in range(1, num_carry_out_bits + 1):
                if i + j < n_p+n_q:
                    s[i, i + j] = sympy.symbols(f's_{i}_{i+j}')

    return n_p, n_q, p, q, s

def generate_column_clauses(N, n_p, n_q, p, q, s):

    """
    Generates the initial symbolic column clauses for the factorization of N.

    Args:
        N (int): The integer to be factored.
        n_p (int): Number of bits for P.
        n_q (int): Number of bits for Q.
        p (dict): Dictionary of symbolic variables for P.
        q (dict): Dictionary of symbolic variables for Q.
        s (dict): Dictionary of symbolic carry variables.
   Returns:
        list: A list of SymPy expressions, one for each column clause C_i.
    """
    n_m = n_p+n_q
    N_bits = [int(bit) for bit in bin(N)[2:].zfill(n_m)]
    N_bits.reverse() # LSB first
    clauses =[]
    # Column 0 is trivial as p_0=q_0=1 and N must be odd (N_0=1)
    # C_0 = N_0 - p_0*q_0 = 1 - 1*1 = 0
    for i in range(1, n_m):
        # 1. Start with the i-th bit of N
        clause = N_bits[i]

        # 2. Subtract product terms p_j * q_{i-j}
        for j in range(n_p):
            if 0 <= i - j < n_q:
                clause -= p[j] * q[i - j]

        # 3. Subtract input carries s_{k,i}
        for k in range(1, i):
            if s[k, i]!= 0:
                clause -= s[k, i]

        # 4. Add output carries 2^j * s_{i,i+j}
        # The value of a carry from column i to i+j is 2^j relative to column i
        j = 1
        while s[i, i + j]!= 0:
            clause += (2**j) * s[i, i + j]
            if i+j>n_p+n_q-1: ##Just in cases where initialize variables is not used and s[i,i+j] is present where i+j>np+nq-1
              clauses.append(s[i,i+j])
            j += 1
        clauses.append(clause)

    return clauses

In [ ]:
def apply_rule_1_and_2(clause):
    """Applies Rule 1 (sum = n) and Rule 2 (sum = 0)."""
    new_constraints = {}
    if not isinstance(clause, sympy.Expr):
       print("Erroneous input")
       return new_constraints

    # Isolate variables and constant term
    terms = clause.as_coefficients_dict()
    const = terms.pop(1, 0)

    # Check for sum = n (Rule 1)
    if all(c == -1 for c in terms.values()) and const == len(terms):
        for var in terms:
            new_constraints[var] = 1
    if all(c==1 for c in terms.values()) and -const == len(terms):
            for var in terms:
              new_constraints[var] = 1
    # Check for sum = 0 (Rule 2)
    if all(c == 1 for c in terms.values()) and const == 0:
        for var in terms:
            new_constraints[var] = 0
    if all(c == -1 for c in terms.values()) and const == 0:
        for var in terms:
            new_constraints[var] = 0

    return new_constraints

def apply_rule_3(clause):
    """Applies Rule 3 (x1 + x2 = 2*x3 => x1=x2=x3). Prioritizes substituting 's' variables."""
    new_constraints = {}
    x1, x2, x3 = sympy.Wild('x1'), sympy.Wild('x2'), sympy.Wild('x3')
    pattern = x1 + x2 - 2*x3
    mul=[]
    mul_ass={}
    ind=0

    match = clause.match(pattern)
    if match:
        # Extract the variables and handle potential non-symbolic matches
        v1 = match[x1]
        v2 = match[x2]
        v3 = match[x3]
        for v in [v1, v2, v3]:
            dih=v.as_coefficients_dict()
            i=0
            for key,value in dih.items():
                if value!=1 or i==1:
                    ind+=1
                    break
                i+=1
            if ind==1:
              break
            if v.is_Mul:
                  mul.append(v)#contains terms like p1*q1 and so on have to be treated seperately

        if ind==1:
          clause=-clause
          mul=[]
          match = clause.match(pattern)
          if match:
              # Extract the variables and handle potential non-symbolic matches
              v1 = match[x1]
              v2 = match[x2]
              v3 = match[x3]
              for v in [v1, v2, v3]:
                  dih=v.as_coefficients_dict()
                  i=0
                  for key,value in dih.items():
                      if value!=1 or i==1:
                          return new_constraints,mul_ass
                      i+=1
                  if v.is_Mul:
                        mul.append(v)#contains terms like p1*q1 and so on have to be treated seperatel

        # Rule 3 implies v1 = v2 = v3.
        # Prioritize substituting 's' variables.
        # If any variable is 's', substitute it with one of the others.
        s_vars = [v for v in [v1, v2, v3] if 's_' in str(v)]
        other_vars = [v for v in [v1, v2, v3] if v not in s_vars]
        if len(mul)==0:
            if s_vars:
                # Substitute all s_vars with the first other_var, or the first s_var if no other_vars
                sub_target = other_vars[0] if other_vars else s_vars[0]
                for s_var in s_vars:
                    if s_var != sub_target:
                        new_constraints[s_var] = sub_target
                # If there are other_vars, ensure they are also equal
                if len(other_vars) > 1:
                    new_constraints[other_vars[1]] = other_vars[0] # Assuming at most 2 other_vars in this rule
            else:
                # If no 's' variables, all are p/q, set them equal
                new_constraints[v1] = v2
                new_constraints[v3] = v2

        else:
          if len(s_vars)==2:
            new_constraints[s_vars[0]] = s_vars[1]
            mul_ass[s_vars[0]]=mul[0]
            mul_ass[s_vars[1]]=mul[0]
          elif len(s_vars)==1:
            if other_vars[0].is_Symbol:
                new_constraints[s_vars[0]] = other_vars[0]
                mul_ass[other_vars[1]]=other_vars[0]
            elif other_vars[1].is_Symbol:
                new_constraints[s_vars[0]] = other_vars[1]
                mul_ass[other_vars[0]]=other_vars[1]
            else:
              mul_ass[s_vars[0]]=other_vars[0]
              mul_ass[other_vars[1]]=other_vars[0]
          else:

            if len(mul)==3:
              mul_ass[other_vars[0]]=other_vars[2]
              mul_ass[other_vars[1]]=other_vars[2]

            elif len(mul)==2:
              if other_vars[0].is_Symbol:
                mul_ass[other_vars[1]]=other_vars[0]
                mul_ass[other_vars[2]]=other_vars[0]
              elif other_vars[1].is_Symbol:
                mul_ass[other_vars[0]]=other_vars[1]
                mul_ass[other_vars[2]]=other_vars[1]
              else:
                mul_ass[other_vars[0]]=other_vars[2]
                mul_ass[other_vars[1]]=other_vars[2]

            else:
                if other_vars[0].is_Mul:
                  mul_ass[other_vars[0]]=other_vars[1]
                  #mul_ass[other_vars[2]]=other_vars[1]
                  new_constraints[other_vars[2]]=other_vars[1]
                elif other_vars[1].is_Mul:
                  mul_ass[other_vars[1]]=other_vars[0]
                  #mul_ass[other_vars[2]]=other_vars[0]
                  new_constraints[other_vars[2]]=other_vars[0]
                else:
                  #mul_ass[other_vars[0]]=other_vars[1]
                  mul_ass[other_vars[2]]=other_vars[1]
                  new_constraints[other_vars[0]]=other_vars[1]


        return new_constraints,mul_ass

    return new_constraints,mul_ass

def apply_rule_4(clause):
    """Applies Rule 4, e.g., x1+x2 = 2*x3+1 => x3=0."""
    new_constraints = {}
    # Specific case from paper: x1 + x2 - 2*x3 - 1 = 0 => x3 = 0
    """x1, x2, x3 = sympy.Wild('x1'), sympy.Wild('x2'), sympy.Wild('x3')
    pattern = x1 + x2 - 2*x3 - 1
    match = clause.match(pattern)
    if match and match[x1].is_Symbol and match[x2].is_Symbol and match[x3].is_Symbol:
            new_constraints[match[x3]] = 0
            return new_constraints
    else:
      clause=-clause
      match = clause.match(pattern)
      if match and match[x1].is_Symbol and match[x2].is_Symbol and match[x3].is_Symbol:
            new_constraints[match[x3]] = 0
            return new_constraints"""

    # General case
    terms = clause.as_coefficients_dict()
    const = terms.pop(1, 0)

    # Check for c_i > n - c_0 (from Eq. 11)
    # Reformulated for clause = 0: c_0 + sum(c_i*y_i) - sum(x_i) = 0
    # Let positive terms be y's and negative terms be x's
    pos_terms = {v: c for v, c in terms.items() if c > 0}
    neg_terms = {v: c for v, c in terms.items() if c < 0}
    pos_sum = sum(pos_terms.values())
    neg_sum = -1*sum(neg_terms.values())
    for y, c_y in pos_terms.items():
            if c_y > neg_sum-const:
              new_constraints[y] = 0
            if c_y>pos_sum+const:
              new_constraints[y] = 1

    for x, c_x in neg_terms.items():
            if -c_x > pos_sum+const:
              new_constraints[x] = 0
            if -c_x>neg_sum-const:
              new_constraints[x] = 1

    return new_constraints

def apply_rule_6(clause):###Changed
  new_constraints = {}
  # General case
  terms = clause.as_coefficients_dict()
  const = terms.pop(1, 0)

  # Check for c_i > n - c_0 (from Eq. 11)
  # Reformulated for clause = 0: c_0 + sum(c_i*y_i) - sum(x_i) = 0
  # Let positive terms be y's and negative terms be x's

  pos_terms = {v: c for v, c in terms.items() if c > 0}
  neg_terms = {v: -c for v, c in terms.items() if c < 0}
  pos_sort = sorted(pos_terms.items(), key=lambda item: item[1], reverse=True)
  neg_sort = sorted(neg_terms.items(), key=lambda item: item[1], reverse=True)
  pos_sum = sum(pos_terms.values())
  neg_sum = sum(neg_terms.values())

  if pos_sum > neg_sum-const and const<0:
    for i in range(len(pos_sort)):
      y, c_y = pos_sort[i]
      for j in range(i+1, len(pos_sort)):
        x, c_x = pos_sort[j]
        k=0
        if c_y+c_x > neg_sum-const:
          if -const-pos_sum+c_y+c_x>0:##Can optimise the equality part

            if 's' in str(y):
              new_constraints[y] = 1-x
            elif 's' in str(x):
              new_constraints[x] = 1-y
            else:
              new_constraints[y]=1-x
            break    ####Have to optimise here###
        else:
          k+=1
          break
      if k==1:
        break

  if neg_sum > pos_sum+const and const>0:
    for i in range(len(neg_sort)):
      y, c_y = neg_sort[i]


      for j in range(i+1, len(neg_sort)):

        x, c_x = neg_sort[j]

        k=0
        if c_y+c_x > pos_sum+const:
          if neg_sum-const-c_y-c_x<0:##Can optimise the equality part
            if 's' in str(y):
              new_constraints[y] = 1-x
            elif 's' in str(x):
              new_constraints[x] = 1-y
            else:
              new_constraints[y]=1-x
            break
        else:
          k+=1
          break
      if k==1:
        break

  return new_constraints

def apply_parity_rule(clause):
    """Applies the Parity Rule."""
    new_constraints = {}
    if not isinstance(clause, sympy.Expr):
        return new_constraints

    terms = clause.as_coefficients_dict()

    odd_terms_sum = sympy.sympify(0)
    for term, coeff in terms.items():
        if coeff % 2!= 0:
            odd_terms_sum += term * (coeff % 2) # Add term with coeff 1 or -1

    # The sum of odd-coefficient terms must have even parity.
    # If odd_terms_sum simplifies to x1 + x2, it implies x1=x2.
    # If it simplifies to x1 + x2 + 1, it implies x1 + x2 = 1.

    # Case 1: x1 + x2 = 0 (implies x1=x2)
    x1, x2 = sympy.Wild('x1'), sympy.Wild('x2')
    match = odd_terms_sum.match(x1 + x2)
    if match and match[x1].is_Symbol and match[x2].is_Symbol:
      if 's' in str(match[x1]):
        new_constraints[match[x1]] = match[x2]
      elif 's' in str(match[x2]):
        new_constraints[match[x2]] = match[x1]
      else:
        new_constraints[match[x1]]=match[x2]
      return new_constraints

    # Case 2: x1 - x2 = 0 (implies x1=x2)
    match = odd_terms_sum.match(x1 - x2)
    if match and match[x1].is_Symbol and match[x2].is_Symbol:
        if 's' in str(match[x1]):
          new_constraints[match[x1]] = match[x2]
        elif 's' in str(match[x2]):
          new_constraints[match[x2]] = match[x1]
        else:
         new_constraints[match[x1]]=match[x2]
        return new_constraints

    # Case 3: x1 + x2 + 1 = 0 (implies x1+x2=1)
    match = odd_terms_sum.match(x1 + x2 + 1)
    if match and match[x1].is_Symbol and match[x2].is_Symbol:
        if 's' in str(match[x1]):
          new_constraints[match[x1]] = 1-match[x2]
        elif 's' in str(match[x2]):
          new_constraints[match[x2]] =1- match[x1]
        else:
         new_constraints[match[x1]]=1-match[x2]
        return new_constraints

    #Case 4:x1+x2-1=0
    match = odd_terms_sum.match(x1 + x2 - 1)
    if match and match[x1].is_Symbol and match[x2].is_Symbol:
        if 's' in str(match[x1]):
          new_constraints[match[x1]] = 1-match[x2]
        elif 's' in str(match[x2]):
          new_constraints[match[x2]] =1- match[x1]
        else:
         new_constraints[match[x1]]=1-match[x2]
        return new_constraints

    return new_constraints

In [ ]:
# @title
def power_rule(expr):
    """
    Reduce a SymPy polynomial assuming all variables are binary (x^n = x for n >= 1).
    """
    expr = sympy.expand(expr)
    new_terms = {}

    for term, coeff in expr.as_coefficients_dict().items():
        if term == 1:
            # constant term
            new_terms[1] = new_terms.get(1, 0) + coeff
        else:
            # collapse powers of variables: x^n -> x
            factors = []
            for symbol, power in term.as_powers_dict().items():
                factors.append(symbol)  # ignore power, since binary
            reduced_term = sympy.Mul(*factors)
            new_terms[reduced_term] = new_terms.get(reduced_term, 0) + coeff

    # reconstruct polynomial
    reduced_expr = sum(coeff * term for term, coeff in new_terms.items())
    return sympy.expand(reduced_expr)

def apply_power_rule(clauses):
    new_clauses = []
    for clause in clauses:
        reduced_clause = power_rule(clause)
        new_clauses.append(reduced_clause)
    return new_clauses

In [ ]:
# @title
def replacement(clauses):
    constraints={}
    """Applies the replacement rule to eliminate variables."""
    for i, clause in enumerate(clauses):
        if clause == 0:
          continue

        # Find variables in the clause
        variables = list(clause.free_symbols)

        # Try to solve for one variable
        for var in variables:
            coeff = clause.as_coefficients_dict().get(var, None)
            if coeff!=1 and coeff!=-1:
              continue
            try:
                # solve() returns a list of solutions
                solutions = sympy.solve(clause, var)

                if len(solutions) == 1:
                    # Found a unique solution for var
                    sol = solutions

                    # Do not replace primary variables (p, q)###############IS THIS NEEDED?################
                    if 's_' in str(var):#Remove this temporarily
                        constraints[var] = sol[0]
                        #clauses[i] = sympy.sympify(0) # This clause is now resolved
                        return constraints # Indicate a change was made

            except Exception:
                continue # Cannot solve for this variable
    return {}

In [ ]:
# @title
def get_qubo_from_clause(clauses):
    # Combine all squared clauses
    hamiltonian = clauses

    # Expand the full expression
    expanded = sympy.expand(hamiltonian)

    # Collect variables
    variables = list(expanded.free_symbols)
    binary_vars = {str(var): Binary(str(var)) for var in variables}

    # Build the Hamiltonian manually
    qubo_ham = 0
    for term, coeff in expanded.as_coefficients_dict().items():
        if term == 1:  # constant term
            qubo_ham += coeff
        else:
            # term may be x, x*y, etc.
            factor = 1
            for symbol in term.free_symbols:
                factor *= binary_vars[str(symbol)]
            qubo_ham += coeff * factor

    return qubo_ham

def qubo_hamiltonian_from_clauses(clauses):
  H=0
  for clause in clauses:
    if clause!=0:
      h=get_qubo_from_clause(clause)
      H+=h*h

  return H

In [ ]:
import math

def common_factor(clause):
    """
    Finds the greatest common divisor of the *integer* coefficients
    in a SymPy expression. Assumes all coefficients are integers.
    """
    if not isinstance(clause, sympy.Expr):
        return 1 # No common factor if not an expression

    terms = clause.as_coefficients_dict()

    # Include the constant term in the coefficients and convert to integers
    # Removed redundant sympify(c), as 'c' is already a SymPy object
    coeffs = [abs(int(c)) for c in terms.values()]

    if not coeffs: # Handle the case of an empty expression (e.g., 0)
        return 1

    # Calculate the GCD of all coefficients
    gcd_val = coeffs[0]
    for i in range(1, len(coeffs)):
        gcd_val = math.gcd(gcd_val, coeffs[i])
        if gcd_val == 1:
            break # Optimization
    if gcd_val==0:
      return 1
    return gcd_val

#Applying substitution to all clauses
def sub_clause(clauses,constraints):
  for i in range(len(clauses)):
    for var,val in constraints.items():
      clauses[i]=clauses[i].subs(var,val)
  return clauses

In [ ]:
#Whole classical pre-processing
def clause_simplifier(initial_clauses):
  clauses=initial_clauses
  assignment_constraints=[]
  expression_constraints=[]
  mul_constraints=[]
  print("Total iterations possible:",2*len(initial_clauses))
  for i in range(2*len(initial_clauses)):
      #print("Clause 1:",clauses[0])
      ind=0
      #Removing common factor
      for i in range(len(clauses)):
            gcd=common_factor(clauses[i])
            clauses[i]=clauses[i]/gcd

      for clause in clauses:
            constraints=apply_rule_1_and_2(clause)
            constraints_={}#Just for using in product=1 process without causing changes to dictionary while iterating over it
            if len(constraints)!=0:
                          for i in constraints.keys():
                            constraints_[i]=constraints[i]
                            if 'p' in str(i) or 'q' in str(i):
                              #print(constraints)
                              assignment_constraints.append({i:constraints[i]})
                              if i.is_Mul and constraints[i]==1:
                                        print("Product=1 detected:",i,constraints[i])
                                        # Extract individual factors from the multiplication term
                                        variables = list(i.free_symbols)
                                        # Assuming the constraint is that the product is 1, which for binary
                                        # variables implies each variable in the product must be 1.
                                        for var in variables:
                                            # Ensure the variable is a Symbol before adding the constraint
                                            if isinstance(var, sympy.Symbol):
                                                constraints_[var]=1
                                                assignment_constraints.append({var: 1})
                          constraints=constraints_
                          ind=1
                          clauses=sub_clause(clauses,constraints)
                          #print("Rule 1,2 applied")
                          #print("Constraint:",constraints)
                          #print("Constraint obtained from:",clause)
                          #print("Simplified clause:",clauses)
                          break


      for clause in clauses:
            constraints=apply_rule_4(clause)
            constraints_={}#Just for using in product=1 process without causing changes to dictionary while iterating over it
            if len(constraints)!=0:
                          for i in constraints.keys():
                            constraints_[i]=constraints[i]
                            if 'p' in str(i) or 'q' in str(i):
                              #print(constraints)
                              assignment_constraints.append({i:constraints[i]})
                              if i.is_Mul and constraints[i]==1:
                                    print("Product=1 detected:",i,constraints[i])
                                    # Extract individual factors from the multiplication term
                                    variables = list(i.free_symbols)
                                    # Assuming the constraint is that the product is 1, which for binary
                                    # variables implies each variable in the product must be 1.
                                    for var in variables:
                                        # Ensure the variable is a Symbol before adding the constraint
                                        if isinstance(var, sympy.Symbol):
                                            constraints_[var]=1
                                            assignment_constraints.append({var: 1})
                          constraints=constraints_
                          ind=1
                          clauses=sub_clause(clauses,constraints)
                          #print("Rule 4 applied")
                          #print("Constraint:",constraints)
                          #print("Constraint obtained from:",clause)
                          #print("Simplified clause:",clauses)
                          break

      for clause in clauses:
            constraints,mul_constraint=apply_rule_3(clause)
            if len(mul_constraint)!=0:
              #print("Mul_Contraint:",mul_constraint)
              mul_constraints.append(mul_constraint)
            if len(constraints)!=0:
              for i in constraints.keys():
                if 'p' in str(i) or 'q' in str(i):
                  #print(constraints)
                  expression_constraints.append({i:constraints[i]})
              ind=1
              clauses=sub_clause(clauses,constraints)
              clauses=apply_power_rule(clauses)
              #print("Rule 3 applied")
              #print("Constraint:",constraints)
              #print("Constraint obtained from:",clause)
              #print("Simplified clause:",clauses)
              break

      for clause in clauses:
            constraints=apply_rule_6(clause)
            if len(constraints)!=0:
              for i in constraints.keys():
                if 'p' in str(i) or 'q' in str(i):
                  #print(constraints)
                  expression_constraints.append({i:constraints[i]})
              ind=1
              #print("Clause applied to:",clause)
              clauses=sub_clause(clauses,constraints)
              clauses=apply_power_rule(clauses)
              #print("Rule 6 applied")
              #print("Constraint:",constraints)
              #print("Constraint obtained from:",clause)
              #print("Simplified clauses:",clauses)
              break

      for clause in clauses:
            constraints=apply_parity_rule(clause)
            if len(constraints)!=0:
              for i in constraints.keys():
                if 'p' in str(i) or 'q' in str(i):
                  #print(constraints)
                  expression_constraints.append({i:constraints[i]})
              ind=1
              clauses=sub_clause(clauses,constraints)
              clauses=apply_power_rule(clauses)
              #print("Rule parity applied")
              #print("Constraint:",constraints)
              #print("Constraint obtained from:",clause)
              #print("Simplified clause:",clauses)

              break

      if ind==0:
        constraints=replacement(clauses)
        if len(constraints)!=0:
              for i in constraints.keys():
                if 'p' in str(i) or 'q' in str(i):
                  #print(constraints)
                  expression_constraints.append({i:constraints[i]})
              clauses=sub_clause(clauses,constraints)
              clauses=apply_power_rule(clauses)
              #print("replaced")
              #print("Constraint:",constraints)
              #print("Simplified clause:",clauses)
        else:
          break
      #print("Number of without replacement iterations:",i)
  return (clauses,assignment_constraints,expression_constraints,mul_constraints)

In [ ]:
def final_steps(model,N,np,nq,addn_constr,addn_ass):
  solv_ind=0
  full_assignment={}
  # The original 'if model!=0:' check causes TypeError when 'model' is a pyqubo.Express object
  # We need to explicitly check if it's the Python integer 0, which would indicate
  # that the Hamiltonian was entirely simplified to zero by classical pre-processing.
  if not (isinstance(model, int) and model == 0):
      model=model.compile()
      bqm=model.to_bqm()
      print("Compiled BQM:", bqm)
      bqm=quadrizate(bqm)
      print("Quadratized BQM has {} non-fixed variables.".format(len(bqm.variables)))

      sampler = SimulatedAnnealingSampler()
      sampleset = sampler.sample(bqm, num_reads=1000)
      best_solution = sampleset.first

      print(f"Energy: {best_solution.energy}")
      print(f"Best solution from sampler:",best_solution.sample)

      # Create a complete assignment dictionary including sampled variables and fixed p_0, q_0
      full_assignment = dict(best_solution.sample)
      solv_ind=1

   #Compile additional assignments
  for ass_dict in addn_ass:
    for i,v in ass_dict.items():
        if sympy.sympify(i).is_Mul:
          continue
        else:
          full_assignment[str(i)]=v # Convert Symbol to string for consistent keys

  initial_len = len(full_assignment)
  #if full ass is null to begin with find random constraint to assign 1
  if solv_ind==0:
      #Assign other constr dependent quantities first
      for _ in range(len(addn_constr) + 5): # Iterate a few times to resolve dependencies
          changes_made = False
          for constr_dict in addn_constr: # Iterate through the list of dictionaries
              # constr_dict is a dictionary like {variable: expression}
              for var, expr in constr_dict.items(): # Get the single item from the dictionary
                  if str(var) not in full_assignment: # Ensure variable name is string for lookup
                      # Try to substitute values from full_assignment into the expression
                      try:
                          # Create a substitution dictionary for sympy
                          subs_dict = {sympy.Symbol(str(k)): v for k, v in full_assignment.items()}
                          substituted_expr = expr.subs(subs_dict)

                          # If the expression simplifies to a numerical value, add it to full_assignment
                          if substituted_expr.is_number:
                              full_assignment[str(var)] = int(substituted_expr) # Assuming binary values
                              changes_made = True
                              #print(f"Resolved {var} = {substituted_expr}")
                      except Exception as e:
                          # Substitution failed, maybe due to unresolved variables in expr
                          #print(f"Could not resolve {var} = {expr}: {e}")
                          pass # Keep trying in the next iteration

          if not changes_made and len(full_assignment) >= initial_len + len(addn_constr):
              # No new variables resolved in this iteration and we've potentially resolved all added constraints
              break


      ind=0
      for const_dict in addn_constr:
        for u,v in const_dict.items():
          if str(u) not in full_assignment.keys(): # Convert Symbol to string for consistent lookup
            full_assignment[str(u)]=1 # Convert Symbol to string for consistent keys
            #print("Assigned 1 to",u)
            ind=1
            break
        if ind==1:
          break


  initial_len = len(full_assignment)
  print(full_assignment)
  # --- START: New, more robust substitution loop ---

    # 1. Convert all constraint dicts to a list of sympy.Eq objects
  all_equations = []
  for constr_dict in addn_constr:
      for var, expr in constr_dict.items():
          # Create a true equation: var = expr
          all_equations.append(sympy.Eq(var, expr))

  # 2. Iteratively solve the system
  # We still loop to allow for multi-step dependencies (e.g., solve A, use A to solve B)
  for _ in range(len(all_equations) + 5): # Iterate enough times to resolve
      changes_made = False
      # Create the substitution dictionary from all currently known values
      subs_dict = {sympy.Symbol(str(k)): v for k, v in full_assignment.items()}

      for eq in all_equations:
          # Substitute all known values into this equation
          substituted_eq = eq.subs(subs_dict)

          # Get all remaining (unknown) variables in this equation
          unknowns = substituted_eq.free_symbols

          # Check if the equation now has exactly ONE unknown variable
          if len(unknowns) == 1:
              unknown_var = list(unknowns)[0]

              # If we haven't already solved for this variable, solve it
              if str(unknown_var) not in full_assignment:
                  try:
                      # Use sympy.solve() to find the value of the unknown
                      solution = sympy.solve(substituted_eq, unknown_var)

                      # Check if solve found a single, numerical solution
                      if solution and isinstance(solution, list) and len(solution) == 1 and solution[0].is_number:

                          solved_value = int(solution[0])
                          full_assignment[str(unknown_var)] = solved_value
                          changes_made = True

                          # IMPORTANT: Update subs_dict *immediately* # This allows the next equation in *this* pass to use this new value
                          subs_dict[unknown_var] = solved_value

                          # print(f"Solved: {unknown_var} = {solved_value}") # <-- Good for debugging

                  except Exception as e:
                      # print(f"Could not solve for {unknown_var} in {substituted_eq}: {e}")
                      pass # Skip if solving fails (e.g., non-linear, no solution)

      # If no progress was made in a full pass, we're done
      if not changes_made:
          break

    # --- END: New substitution loop ---

  print("After trying to solve:",full_assignment)
  P = 0
  Q = 0

  # Reconstruct P and Q from the resolved bits using original n_p and n_q
  # Initialize with fixed bits based on initialize_variables logic (p_0=1, p_{n_p-1}=1, q_0=1, q_{n_q-1}=1)
  # Ensure these are included in full_assignment or handled here if they aren't variables in the BQM
  # Based on initialize_variables, p_0, p_{n_p-1}, q_0, q_{n_q-1} are Symbols.
  # We need to get their resolved values from full_assignment.

  p_bits = {}
  q_bits = {}

  # Populate p_bits and q_bits from full_assignment
  for var_name, value in full_assignment.items():
      if 'p_' in var_name:
          try:
              bit_pos = int(var_name.split('_')[1])
              p_bits[bit_pos] = value
          except ValueError:
              pass # Not a standard p_i variable
      elif 'q_' in var_name:
          try:
              bit_pos = int(var_name.split('_')[1])
              q_bits[bit_pos] = value
          except ValueError:
              pass # Not a standard q_i variable

  # Ensure fixed bits are in the dictionaries, using 1 as default if not resolved
  # (This might need refinement if fixed bits can be resolved to 0 based on constraints)
  # We are assuming p_0, q_0, p_np-1, q_nq-1 are 1 as per initialize_variables.
  # If they are part of full_assignment due to being variables in the BQM,
  # their values from full_assignment will be used. Otherwise, default to 1.
  if 0 not in p_bits:
      p_bits[0] = 1
  if np-1 not in p_bits:
      p_bits[np-1] = 1
  if 0 not in q_bits:
      q_bits[0] = 1
  if nq-1 not in q_bits:
      q_bits[nq-1] = 1


  # Reconstruct P and Q by summing all bits with their power of 2
  P = 0
  for bit_pos in range(np):
      if bit_pos in p_bits:
          P += int(p_bits[bit_pos]) * (2 ** bit_pos)
      # Else: bit value is not resolved, assume 0 or handle as needed
      # For binary factoring, unresolved bits can sometimes be inferred as 0 or 1 based on other constraints
      # For now, we only sum explicitly assigned bits.

  Q = 0
  for bit_pos in range(nq):
      if bit_pos in q_bits:
          Q += int(q_bits[bit_pos]) * (2 ** bit_pos)
      # Else: bit value is not resolved

  # Convert P and Q to standard Python integers for verification
  P_int = int(P)
  Q_int = int(Q)

  #print("P_vars:",p_bits) # Use p_bits and q_bits for printing
  #print("Q_vars:",q_bits)

  print(f"Factors of {N} are: {P_int}, {Q_int}")
  if P_int * Q_int == N:
      print(f"Verification: {P_int} * {Q_int} = {P_int*Q_int} (Correct)")
  else:
      print(f"Verification: {P_int} * {Q_int} = {P_int*Q_int} (Incorrect)")

In [ ]:
#N has to be a bi-prime with factors being equal bit length for proper factorization
def factorize2(N):
  n_p,n_q,p,q,s=initialize_variables(N)
  initial_clauses=generate_column_clauses(N, n_p, n_q, p, q, s)
  clauses,ass,expr,mul=clause_simplifier(initial_clauses)
  H=qubo_hamiltonian_from_clauses(clauses)
  final_steps(H,N,n_p,n_q,expr,ass)

In [ ]:
def get_qubo_matrix2(N):
    n_p, nq, p, q, s = initialize_variables(N)
    initial_clauses = generate_column_clauses(N, n_p, nq, p, q, s)
    clauses, ass, expr, mul = clause_simplifier(initial_clauses)
    model = qubo_hamiltonian_from_clauses(clauses)
    model = model.compile()
    bqm = model.to_bqm()
    bqm = quadrizate(bqm)

    print("Compiled BQM:", bqm)
    print("Quadratized BQM has {} non-fixed variables.".format(len(bqm.variables)))

    qubo_dict, offset = bqm.to_qubo()

    # --- FIX: SORT VARIABLES TO ENSURE STABLE ORDERING ---
    # We sort by string representation to ensure deterministic mapping
    sorted_variables = sorted(list(bqm.variables), key=str)

    variable_to_index = {v: i for i, v in enumerate(sorted_variables)}
    index_to_variable = {i: v for v, i in variable_to_index.items()}
    # -----------------------------------------------------

    size = len(sorted_variables)
    qubo_matrix = np.zeros((size, size))

    neighbors = defaultdict(list)   # adjacency list
    h = np.zeros(N, dtype=np.float32)
    offset = 0.0

    for (u, v), value in qubo_dict.items():
        if u in variable_to_index and v in variable_to_index:
            i = variable_to_index[u]
            j = variable_to_index[v]
            qubo_matrix[i, j] = value

    print(f"qubo_dict is {qubo_dict}")

    return qubo_matrix, index_to_variable, ass, expr, mul

In [ ]:
def post_processing2(sol,N,ass_constr,addn_constr,index_to_variable):#Assuming the solution from SPIM is given as a list of 0s and 1s and sol[i] represents the solution of the ith index of qubo matrix
      full_assignment = {}
      for ass_dict in ass_constr:
        for i,v in ass_dict.items():
          if sympy.sympify(i).is_Mul:
            continue
          else:
            full_assignment[str(i)]=v # Convert Symbol to string for consistent keys



      for i in range(len(sol)):
        full_assignment[index_to_variable[i]]=sol[i]

      initial_len = len(full_assignment)
    # Simple iterative substitution loop for addn_constr (can be improved for complex dependencies)
      for _ in range(len(addn_constr) + 5): # Iterate a few times to resolve dependencies
          changes_made = False
          for constr_dict in addn_constr: # Iterate through the list of dictionaries
              # constr_dict is a dictionary like {variable: expression}
              for var, expr in constr_dict.items(): # Get the single item from the dictionary
                  if str(var) not in full_assignment: # Ensure variable name is string for lookup
                      # Try to substitute values from full_assignment into the expression
                      try:
                          # Create a substitution dictionary for sympy
                          subs_dict = {sympy.Symbol(str(k)): v for k, v in full_assignment.items()}
                          substituted_expr = expr.subs(subs_dict)

                          # If the expression simplifies to a numerical value, add it to full_assignment
                          if substituted_expr.is_number:
                              full_assignment[str(var)] = int(substituted_expr) # Assuming binary values
                              changes_made = True
                              #print(f"Resolved {var} = {substituted_expr}")
                      except Exception as e:
                          # Substitution failed, maybe due to unresolved variables in expr
                          #print(f"Could not resolve {var} = {expr}: {e}")
                          pass # Keep trying in the next iteration

          if not changes_made and len(full_assignment) >= initial_len + len(addn_constr):
              # No new variables resolved in this iteration and we've potentially resolved all added constraints
              break


      # Manually add p_0 and q_0 if they are not present and needed for factor calculation
      # This requires knowing n_p and n_q from initialization
      # For now, let's assume they are always 1 unless determined otherwise by constraints
      # A better way might be to include them in the initial variable set and handle them in constraints
      # based on the problem definition (p_0=1, q_0=1).

      # Calculate the factors P and Q
      P = 0
      Q = 0

      # Need to know the original variable names and their bit positions (0 to n_p-1, 0 to n_q-1)
      # This information should ideally be passed from initialize_variables

      # As a workaround, iterate through resolved variables and check if they are p or q bits
      # This assumes the variable naming convention 'p_i' and 'q_i'
      p_vars = {}
      q_vars = {}

      for var_name, value in full_assignment.items():
          if 'p_' in var_name:
              try:
                  bit_pos = int(var_name.split('_')[1])
                  p_vars[bit_pos] = value
              except ValueError:
                  pass # Not a standard p_i variable
          elif 'q_' in var_name:
              try:
                  bit_pos = int(var_name.split('_')[1])
                  q_vars[bit_pos] = value
              except ValueError:
                  pass # Not a standard q_i variable

      # Add p_0=1 and q_0=1 if not already present (assuming this is the rule)
      # This is a fallback; ideally, p_0 and q_0 should be part of the variable set and constraints
      # Determine n_p and n_q from the variables found, or pass them in
      # For now, let's assume n_p and n_q can be inferred or are small enough.
      # A more robust solution would pass n_p and n_q from initialize_variables.
      # For N=15 (n_p=3, n_q=3), p_0, p_1, p_2 and q_0, q_1, q_2 are expected.
      # Let's try to infer max bit position to guess n_p, n_q
      #max_p_bit = max(p_vars.keys(), default=-1)
      #max_q_bit = max(q_vars.keys(), default=-1)
      #inferred_n_p = max_p_bit + 1
      #inferred_n_q = max_q_bit + 1

      nq=ceil(0.5*log2(N))
      np=nq

      if 0 not in p_vars:
          p_vars[0] = 1
      if 0 not in q_vars:
          q_vars[0] = 1

      if np-1 not in p_vars:
          p_vars[np-1] = 1

      if nq-1 not in q_vars:
          q_vars[nq-1] = 1

      # Ensure all expected bit positions for p and q have values (either sampled or from constraints)
      # If not, it means the constraints or sampling did not provide a full solution
      # This part might need refinement based on how missing variables should be handled
      # (e.g., assuming 0 if not determined, which is not always correct for binary)
      # For now, we'll proceed with available values.


      # Reconstruct P and Q from the resolved bits
      # Use sorted keys to ensure correct bit position weighting
      P = 0  # Initialize P as a standard Python integer
      Q = 0  # Initialize Q as a standard Python integer

      for bit_pos in sorted(p_vars.keys()):
          P += int(p_vars[bit_pos]) * (2 ** bit_pos)

      for bit_pos in sorted(q_vars.keys()):
          Q += int(q_vars[bit_pos]) * (2 ** bit_pos)

      # Convert P and Q to standard Python integers for verification
      P_int = int(P)
      Q_int = int(Q)

      print(f"Factors of {N} are: {P_int}, {Q_int}")
      if P_int * Q_int == N:
          print(f"Verification: {P_int} * {Q_int} = {P_int*Q_int} (Correct)")
      else:
          print(f"Verification: {P_int} * {Q_int} = {P_int*Q_int} (Incorrect)")

In [ ]:
def solve_qubo_from_matrix(qubo_matrix, num_reads=10):
    """
    Solves a QUBO problem defined by a NumPy matrix using Simulated Annealing.

    Args:
        qubo_matrix (np.array): The QUBO matrix (can be upper triangular or symmetric).
        num_reads (int): Number of times to run the annealing process (more reads = higher chance of finding global min).

    Returns:
        best_solution (dict): The qubit configuration with the lowest energy.
        best_energy (float): The energy of that configuration.
        all_samples (SampleSet): The full results object.
    """

    # 1. Convert NumPy matrix to the dictionary format expected by D-Wave/Neal
    # Format: Q = {(qubit_i, qubit_j): bias, ...}
    Q = {}
    rows, cols = qubo_matrix.shape

    for i in range(rows):
        for j in range(cols):
            # We iterate the whole matrix. If you have a symmetric matrix,
            # the solver will automatically sum (i,j) and (j,i) interactions.
            if qubo_matrix[i, j] != 0:
                Q[(i, j)] = qubo_matrix[i, j]

    # 2. Initialize the Simulated Annealing Sampler
    sampler = SimulatedAnnealingSampler() # Corrected: Use SimulatedAnnealingSampler from dwave.samplers

    # 3. Run the solver
    # sample_qubo handles the annealing process
    sampleset = sampler.sample_qubo(Q, num_reads=num_reads)

    # 4. Extract the best solution (lowest energy)
    best_sample = sampleset.first.sample
    best_energy = sampleset.first.energy

    return best_sample, best_energy, sampleset


In [ ]:
import numpy as np
import pandas as pd


def qubo_to_ising(Q, N):
    """
    Converts a symmetric QUBO matrix Q to Ising parameters J, h, and offset.
    Mapping: q = (x + 1) / 2  => (-1 -> 0, 1 -> 1)
    """
    Q = np.array(Q)

    # 1. Calculate J matrix (J_ij = Q_ij / 4, diagonal = 0) for DA2
    # 2. Calculate J matrix (J_ij = Q_ij / 2, diagonal = 0) for DA1
    J = Q / 4.0
    np.fill_diagonal(J, 0)

    # 2. Calculate h vector (h_i = 0.5 * row_sum(Q))
    h = 0.5 * np.sum(Q, axis=1)

    # 3. Calculate constant offset
    offset = 0.25 * (np.sum(Q) + np.trace(Q))

    # Save J to CSV: no index, no headers
    pd.DataFrame(J).to_csv(f'/content/drive/MyDrive/J_matrix_{N}.csv', index=False, header=False)

    # Save h to CSV: row 1, no index, no headers
    pd.DataFrame(h.reshape(1, -1)).to_csv(f'/content/drive/MyDrive/h_matrix_{N}.csv', index=False, header=False)

    return J, h, offset

In [ ]:
import time
# P, Q = 120011, 180307
# N=P*Q
N = 10403

qubo_matrix,ind_to_var,ass,expr,mul=get_qubo_matrix2(N)
#print("Qubo Matrix:")
#print(qubo_matrix)
print("Assignment constraints:",ass)
print("Expression constraints:",expr)
start_time = time.time()
best_sample, best_energy, sampleset = solve_qubo_from_matrix(qubo_matrix)
end_time = time.time()
print(f"Time taken to solve: {end_time - start_time} seconds")
print(f"\nLowest Energy: {best_energy}")
print("Best sample",best_sample)
solution_list = [best_sample[i] for i in range(len(qubo_matrix))]
#print(f"\nSolution List: {solution_list}")

post_processing2(solution_list,N,ass,expr,ind_to_var)

Factoring N = 10403 (14 bits)
Assuming n_p = 7, n_q = 7
Total iterations possible: 26
Compiled BQM: BinaryQuadraticModel({'q_3': -12005.0, 'q_4': -22496.0, 'p_5': -20345.0, 'q_5': -20837.0, 'q_1': 0.0, 'q_2': -6137.0, 'w1': 168.0, 'w2': 2688.0, 'w3': 84.0, 'w4': 32.0, 'w#4': 32.0, 'w5': 172.0, 'w7': 360.0, 'w9': 24.0, 'w10': 48.0, 'w11': 192.0, 'w#11': 192.0, 'w12': 924.0, 'w13': 1944.0, 'w14': 3984.0, 'w15': 192.0, 'w#15': 192.0, 'w16': 932.0, 'w17': 1960.0, 'w18': 96.0, 'w19': 344.0, 'w20': 720.0, 'w22': 1024.0, 'w#22': 1024.0, 'w23': 5504.0, 'w24': 11520.0, 'w25': 4016.0, 'w26': 960.0, 'w27': 256.0, 'w#27': 256.0, 'w28': 512.0, 'w#28': 512.0, 'w29': 8.0, 'w#29': 8.0, 'w30': 16.0, 'w#30': 16.0}, {('q_4', 'q_3'): 7980.0, ('p_5', 'q_3'): 13702.0, ('p_5', 'q_4'): 19796.0, ('q_5', 'q_3'): 3362.0, ('q_5', 'q_4'): 8636.0, ('q_5', 'p_5'): 6086.0, ('q_1', 'p_5'): 180.0, ('q_1', 'q_5'): 492.0, ('q_2', 'q_3'): 2730.0, ('q_2', 'q_4'): 3708.0, ('q_2', 'p_5'): 7886.0, ('q_2', 'q_5'): 1454.0, ('w1

In [ ]:
if __name__ == "__main__":

    # Converting Q into a symmetric matrix
    Q = (qubo_matrix + qubo_matrix.T) / 2

    J, h, offset = qubo_to_ising(Q, N)

    print("QUBO Matrix Q:")
    print()

    print("Ising Coupling Matrix J:")
    print(J)
    print()

    print("Ising Linear Field h:")
    # print(h)
    print()

    print(f"Constant Offset: {offset}")
    print()

In [ ]:

s = "1	-1	1	1	-1	1	-1	1	-1	1	-1	-1	1	-1	-1	-1	-1	1	-1	-1	-1	-1	-1	-1	-1	-1	1	1	-1	-1	1	1	-1	-1	-1	-1	-1	-1	1	-1	1	-1	-1	-1	-1	1	1	1	1	-1	-1	-1	-1	1	-1	-1	-1	-1	-1	-1	-1	-1	1	-1	-1	-1	-1	-1	-1	1	-1	1	-1	-1	-1	-1	-1	-1	-1	-1	-1	1	-1	1	1	-1	-1	1	1	-1	-1	1	1	1	-1	1	1	1	-1	1	-1	-1	-1	-1	-1	-1	1	-1	-1	1	-1	-1	-1	-1	-1	-1	1	-1	-1	-1	1	1	1	-1	1	-1	-1	1	-1	-1	-1	1	1	1	-1	-1	1	-1	-1	-1	-1	-1	1	1	1	-1	-1"

s_list = [-1, -1,  1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1,  1, -1, -1,
        1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1, -1, -1,
       -1,  1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,
       -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,
       -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1,
       -1, -1, -1,  1,  1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1,  1, -1,
       -1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1,  1, -1,  1,
        1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1, -1,  1,  1, -1,
       -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1,
        1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1,
       -1, -1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1, -1, -1, -1,  1,
       -1, -1, -1, -1,  1,  1, -1,  1, -1, -1,  1, -1, -1, -1, -1,  1, -1,
        1,  1, -1, -1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1,  1, -1, -1, -1,  1,
        1,  1, -1, -1,  1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1,
        1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1,  1,  1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,
       -1,  1,  1,  1, -1,  1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1,  1,
        1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1,  1, -1,  1, -1, -1, -1, -1,  1, -1, -1,  1, -1, -1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
       -1, -1]
# solution_list = [(int(x)+ 1)//2 for x in s.split()]
# print(solution_list)
# solution_list = [(i + 1) // 2 for i in s_list]
actual_sol = [np.int8(0), np.int8(1), np.int8(1), np.int8(1), np.int8(1), np.int8(1), np.int8(1), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(1), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(1), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(1), np.int8(0), np.int8(1), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(1), np.int8(0), np.int8(0), np.int8(1), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(1), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(1), np.int8(0), np.int8(1), np.int8(1), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(1), np.int8(0), np.int8(0), np.int8(0), np.int8(1), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(1), np.int8(0), np.int8(1), np.int8(0), np.int8(1), np.int8(1), np.int8(0), np.int8(1), np.int8(1), np.int8(0), np.int8(0), np.int8(0), np.int8(1), np.int8(1), np.int8(0), np.int8(1), np.int8(1), np.int8(0), np.int8(0), np.int8(1), np.int8(0), np.int8(0), np.int8(1), np.int8(0), np.int8(1), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(1), np.int8(1), np.int8(1), np.int8(0), np.int8(1), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(1), np.int8(0), np.int8(0), np.int8(0), np.int8(1), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(1), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0)]

solution_list = [(i + 1) // 2 for i in np.load(f'parallel_sa_{N}.npy')]

post_processing2(solution_list,N,ass,expr,ind_to_var)

Factors of 377977 are: 691, 547
Verification: 691 * 547 = 377977 (Correct)
